In [18]:
#### 
    # ZIEL: IMU DATEN auf 100 Hz resamplen damit gleiche Lönge wie Kniewinkel aus RING
    #
    # ABER: funktioniert nicht, deutlich größere array Länge (Wieso?)
    #
    # Frage: gehen bei RING sample rows verloren? -> in RING algo nochmal schauen..
    # Lösung: RING Knee angle rate wurde downsampled zu camera rate: entweder IMU downsamplen oder RING upsamplen

import numpy as np
import os
import math
import qmt
import imt
import pandas as pd
from pathlib import Path

# Load knee angle data
results_path_knee_angle = "../imt-main/examples/results"
ring_angles_kl = np.concatenate(np.load(os.path.join(results_path_knee_angle, "Angles_IMU_RING_Knee_Left.npy"), allow_pickle=True))
ring_angles_kr = np.load(os.path.join(results_path_knee_angle, "Angles_IMU_RING_Knee_Right.npy"), allow_pickle=True)

seq_names = np.load(os.path.join(results_path_knee_angle, "Sequences_names.npy"), allow_pickle=True)

# Load labels
results_path_label = "./results"
labels_extendedLeg_100 = np.load(os.path.join(results_path_label, "labels_extendedLeg_RING_100.npy"), allow_pickle=True)
labels_stiffMovement_100 = np.load(os.path.join(results_path_label, "labels_stiffMovement_RING_100.npy"), allow_pickle=True)

# Load IMU raw data
imu_data_all = []
path_data = "../Daten/"
folder_data_imu = path_data + "Dict_Frames/"
filenames_imu = [str(f) for f in Path(folder_data_imu).rglob('*') if f.is_file()]  # all filnames in array

sequences = pd.read_csv(path_data + "sequences.txt", delimiter="\t", header=None, index_col=0)
seq_imu_start = sequences.iloc[:, 1:2].values.ravel()   # IMU start frame
sensors = ["S0333", "S1094", "S0593", "S0994", "S0477"] 

for i_seq in range(len(filenames_imu)):
    # LOAD DATA
    data_imu_seq = np.load(filenames_imu[i_seq], allow_pickle=True).item()

    # PREPARE DATA
    imu_data = {
        i: dict(acc=data_imu_seq[sensors[i]]["acc"][seq_imu_start[i_seq]:], gyr=data_imu_seq[sensors[i]]["gyr_rad"][seq_imu_start[i_seq]:])
        for i in range(len(sensors))
        }
    imu_data = imt.utils.resample(imt.utils.crop_tail(imu_data, 52), 52, 100)
    imu_data = imt.utils.resample(imt.utils.crop_tail(imu_data, 100), 100, 30)

    imu_data[0] = dict(
        acc=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["acc"]),
        gyr=qmt.rotate(qmt.quatFromAngleAxis(-np.pi, [0, 0, 1]), imu_data[0]["gyr"]),
    )
    imu_data_all.append(imu_data)

`crop_tail`: Crop off at t=177.15384615384616s
`crop_tail`: Crop off at t=177.16s
`crop_tail`: Crop off at t=181.15384615384616s
`crop_tail`: Crop off at t=181.16s
`crop_tail`: Crop off at t=180.6153846153846s
`crop_tail`: Crop off at t=180.62s
`crop_tail`: Crop off at t=199.98076923076923s
`crop_tail`: Crop off at t=199.99s
`crop_tail`: Crop off at t=124.61538461538461s
`crop_tail`: Crop off at t=124.62s
`crop_tail`: Crop off at t=219.07692307692307s
`crop_tail`: Crop off at t=219.08s
`crop_tail`: Crop off at t=184.0s
`crop_tail`: Crop off at t=184.0s
`crop_tail`: Crop off at t=183.30769230769232s
`crop_tail`: Crop off at t=183.31s
`crop_tail`: Crop off at t=199.84615384615384s
`crop_tail`: Crop off at t=199.85s
`crop_tail`: Crop off at t=179.69230769230768s
`crop_tail`: Crop off at t=179.7s
`crop_tail`: Crop off at t=185.92307692307693s
`crop_tail`: Crop off at t=185.93s
`crop_tail`: Crop off at t=180.07692307692307s
`crop_tail`: Crop off at t=180.08s
`crop_tail`: Crop off at t=185.4

In [19]:
S0333_acc = np.vstack([entry[0]['acc'] for entry in imu_data_all])
S0333_gyr = np.vstack([entry[0]['gyr'] for entry in imu_data_all])
S1094_acc = np.vstack([entry[1]['acc'] for entry in imu_data_all])
S1094_gyr = np.vstack([entry[1]['gyr'] for entry in imu_data_all])
S0593_acc = np.vstack([entry[2]['acc'] for entry in imu_data_all])
S0593_gyr = np.vstack([entry[2]['gyr'] for entry in imu_data_all])
S0994_acc = np.vstack([entry[3]['acc'] for entry in imu_data_all])
S0994_gyr = np.vstack([entry[3]['gyr'] for entry in imu_data_all])
S0477_acc = np.vstack([entry[4]['acc'] for entry in imu_data_all])
S0477_gyr = np.vstack([entry[4]['gyr'] for entry in imu_data_all])
imu_data_ml = np.hstack((S0333_acc, S0333_gyr, S1094_acc, S1094_gyr, S0593_acc, S0593_gyr, S0994_acc, S0994_gyr, S0477_acc, S0477_gyr))

In [17]:
path_results = "./results" 
os.makedirs(path_results, exist_ok=True) # check if directory exists
np.save(os.path.join(path_results, "IMU_data_ml.npy"), np.array(imu_data_ml, dtype=object))
np.save(os.path.join(path_results, "IMU_data_all.npy"), np.array(imu_data_all, dtype=object))